1. finnhub에서 가져온 뉴스 데이터 확인

In [10]:
import pandas as pd

df = pd.read_csv("./TSLA_extended_news_2025-06-14.csv")

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9122 entries, 0 to 9121
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 9122 non-null   int64  
 1   title              9122 non-null   object 
 2   summary            8995 non-null   object 
 3   link               9122 non-null   object 
 4   publisher          0 non-null      float64
 5   category           9122 non-null   object 
 6   pubDate            9122 non-null   object 
 7   image              5022 non-null   object 
 8   related            9122 non-null   object 
 9   source             9122 non-null   object 
 10  collection_period  9122 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 784.1+ KB


In [3]:
df.head(5)

,id,title,summary,link,publisher,category,pubDate,image,related,source,collection_period
0,135270013,3 No-Brainer Artificial Intelligence (AI) Grow...,Finding value in artificial intelligence (AI) ...,https://finnhub.io/api/news?id=3e8de88fcb69df9...,NaN,company,2025-06-14 09:55:00,https://s.yimg.com/rz/stage/p/yahoo_finance_en...,AAPL,Yahoo,2025-06-07_2025-06-14
1,135258199,Apple Reroutes 97% Of Foxconn iPhone Exports F...,Apple Inc. (NASDAQ:AAPL) has reportedly increa...,https://finnhub.io/api/news?id=e6e09272a3b910a...,NaN,company,2025-06-14 01:30:33,https://s.yimg.com/rz/stage/p/yahoo_finance_en...,AAPL,Yahoo,2025-06-07_2025-06-14
2,135270015,Why Superintelligent AI Isn’t Taking Over Anyt...,"Despite claims from top names in AI, researche...",https://finnhub.io/api/news?id=d8a8fdff582d1f1...,NaN,company,2025-06-14 01:00:00,https://s.yimg.com/rz/stage/p/yahoo_finance_en...,AAPL,Yahoo,2025-06-07_2025-06-14
3,135257945,Why Apple (AAPL) Dipped More Than Broader Mark...,Apple (AAPL) reached $196.45 at the closing of...,https://finnhub.io/api/news?id=6964bc22b1f8f8c...,NaN,company,2025-06-13 21:45:02,https://s.yimg.com/rz/stage/p/yahoo_finance_en...,AAPL,Yahoo,2025-06-07_2025-06-14
4,135257921,Apple's $4.4B India Play: The Secret iPhone Sh...,"As Trump targets China, Apple quietly reroutes...",https://finnhub.io/api/news?id=7f50a3430d3f330...,NaN,company,2025-06-13 21:06:46,https://s.yimg.com/rz/stage/p/yahoo_finance_en...,AAPL,Yahoo,2025-06-07_2025-06-14


In [4]:
df['title'][0]

'GameStop earnings, US-China talks, May CPI: What to Watch'

In [5]:
df['summary'][0]

"Market Domination Overtime hosts Julie Hyman and Josh Lipton preview the top market stories and headlines Wall Street will be paying attention to next week, including corporate earnings from companies like GameStop (GME), Oracle (ORCL), Adobe (ADBE), and the J.M. Smucker Company (SJM); Apple's (AAPL) 2025 Worldwide Developers Conference (WWDC) kicking off; May's inflation data; and the launch of Tesla's (TSLA) robotaxi beta in Austin, Texas, starting on June 12. To watch more expert insights and analysis on the latest market action, check out more Market Domination Overtime\xa0here."

In [9]:
df['pubDate'][0]


'2025-06-06 23:00:00'

In [11]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

MODEL = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, return_all_scores=True)

df = pd.read_csv("AAPL_news_2025-06-07.csv") 
df = df.fillna("")

df["text"] = df["title"] + ". " + df["summary"]
texts = df["text"].tolist()

results = finbert(texts, batch_size=8)

pos_scores, neu_scores, neg_scores = [], [], []

for res in results:
    d = {r["label"].lower(): r["score"] for r in res}
    pos_scores.append(d.get("positive", 0.0))
    neu_scores.append(d.get("neutral", 0.0))
    neg_scores.append(d.get("negative", 0.0))

df["finbert_positive"] = pos_scores
df["finbert_neutral"] = neu_scores
df["finbert_negative"] = neg_scores

output_csv = "apple_finbert_finnhub.csv"
df.to_csv(output_csv, index=False)
print(f"결과 저장 완료: {output_csv}")


Device set to use cuda:0
c:\Users\jhh33\anaconda3\envs\pythonProject1\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


✅ 결과 저장 완료: apple_finbert_finnhub.csv
